# Homework 1 (Total Points: 250) <a class="anchor" id="top"></a>


**Submission instructions**:
- The cells with the `# YOUR CODE HERE` denote that these sections are graded and you need to add your implementation.
- For Part 1: You can use the `nltk`, `NumPy`, and `matplotlib` libraries here. Other libraries, e.g., `gensim` or `scikit-learn`, may not be used. For Part 2: `gensim` is allowed in addition to the imported libraries in the next code cell
- Please use Python 3.6.5 and `pip install -r requirements.txt` to avoid version issues.
- The notebook you submit has to have the student ids, separated by underscores (E.g., `12341234_12341234_12341234_hw1.ipynb`).
- This will be parsed by a regexp, **so please double check your filename**.
- Only one member of each group has to submit the file (**please do not compress the .ipynb file when you will submit it**) to canvas.
- **Make sure to check that your notebook runs before submission**. A quick way to do this is to restart the kernel and run all the cells.  
- Do not change the number of arugments in the given functions.
- **Please do not delete/add new cells**. Removing cells **will** lead to grade deduction. 
- Note, that you are not allowed to use Google Colab.


**Learning Goals**:
- [Part 1, Term-based matching](#part1) (165 points):
    - Learn how to load a dataset and process it.
    - Learn how to implement several standard IR methods (TF-IDF, BM25, QL) and understand their weaknesses & strengths.
    - Learn how to evaluate IR methods.
- [Part 2, Semantic-based matching](#part2) (85 points):
    - Learn how to implement vector-space retrieval methods (LSI, LDA).
    - Learn how to use LSI and LDA for re-ranking.

    
**Resources**: 
- **Part 1**: Sections 2.3, 4.1, 4.2, 4.3, 5.3, 5.6, 5.7, 6.2, 7, 8 of [Search Engines: Information Retrieval in Practice](https://ciir.cs.umass.edu/downloads/SEIRiP.pdf)
- **Part 2**: [LSI - Chapter 18](https://nlp.stanford.edu/IR-book/pdf/18lsi.pdf) from [Introduction to Information Retrieval](https://nlp.stanford.edu/IR-book/) book and the [original LDA paper](https://jmlr.org/papers/volume3/blei03a/blei03a.pdf)

In [1]:
# imports 
# TODO: Ensure that no additional library is imported in the notebook. 
# TODO: Only the standard library and the following libraries are allowed:
# TODO: You can also use unlisted classes from these libraries or standard libraries (such as defaultdict, Counter, ...).

import os
import zipfile
from functools import partial

import nltk
import requests
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

from ipywidgets import widgets
from IPython.display import display, HTML
#from IPython.html import widgets
from collections import namedtuple

%matplotlib inline


# Part 1: Term-based Matching (165 points) <a class="anchor" id="part1"></a>

[Back to top](#top)

In the first part, we will learn the basics of IR from loading and preprocessing the material, to implementing some well known search algorithms, to evaluating the ranking performance of the implemented algorithms. We will be using the CACM dataset throughout the assignment. The CACM dataset is a collection of titles and abstracts from the journal CACM (Communication of the ACM).

Table of contents:
- [Section 1: Text Processing](#text_processing) (5 points)
- [Section 2: Indexing](#indexing) (10 points)
- [Section 3: Ranking](#ranking) (80 points)
- [Section 4: Evaluation](#evaluation) (40 points)
- [Section 5: Analysis](#analysis) (30 points)


---
## Section 1: Text Processing (5 points)<a class="anchor" id="text_processing"></a>

[Back to Part 1](#part1)

In this section, we will load the dataset and learn how to clean up the data to make it usable for an IR system. 
First, go through the implementation of the following functions:
- `read_cacm_docs`: Reads in the CACM documents.
- `read_queries`: Reads in the CACM queries.
- `load_stopwords`: Loads the stopwords.

The points of this section are earned for the following implementations:
- `tokenize` (3 points): Tokenizes the input text.
- `stem_token` (2 points): Stems the given token. 

We are using the [CACM dataset](http://ir.dcs.gla.ac.uk/resources/test_collections/cacm/), which is a small, classic IR dataset, composed of a collection of titles and abstracts from the journal CACM. It comes with relevance judgements for queries, so we can evaluate our IR system. 


---
### 1.1 Read the CACM documents


The following cell downloads the dataset and unzips it to a local directory.

In [2]:
def download_dataset():
    folder_path = os.environ.get("IR1_DATA_PATH")
    if not folder_path:
        folder_path = "./datasets/"
    os.makedirs(folder_path, exist_ok=True)
    
    file_location = os.path.join(folder_path, "cacm.zip")
    
    # download file if it doesn't exist
    if not os.path.exists(file_location):
        
        url = "https://surfdrive.surf.nl/files/index.php/s/M0FGJpX2p8wDwxR/download"

        with open(file_location, "wb") as handle:
            print(f"Downloading file from {url} to {file_location}")
            response = requests.get(url, stream=True)
            for data in tqdm(response.iter_content()):
                handle.write(data)
            print("Finished downloading file")
    
    if not os.path.exists(os.path.join(folder_path, "train.txt")):
        
        # unzip file
        with zipfile.ZipFile(file_location, 'r') as zip_ref:
            zip_ref.extractall(folder_path)
        
download_dataset()

---

You can see a brief description of each file in the dataset by looking at the README file:

In [3]:
##### Read the README file 
with open ("./datasets/README","r") as file:
    readme = file.read()
    print(readme)
#####

Files in this directory with sizes:
          0 Jun 19 21:01 README

    2187734 Jun 19 20:55 cacm.all              text of documents
        626 Jun 19 20:58 cite.info             key to citation info
                                                (the X sections in cacm.all)
       2668 Jun 19 20:55 common_words           stop words used by smart
       2194 Jun 19 20:55 make_coll*             shell script to make collection
       1557 Jun 19 20:55 make_coll_term*        ditto (both useless without
                                                smart system)
       9948 Jun 19 20:55 qrels.text             relation giving
                                                    qid did 0 0
                                                to indicate dument did is
                                                relevant to query qid
      13689 Jun 19 20:55 query.text             Original text of the query



---
We are interested in 4 files:
- `cacm.all` : Contains the text for all documents. Note that some documents do not have abstracts available
- `query.text` : The text of all queries
- `qrels.text` : The relevance judgements
- `common_words` : A list of common words. This may be used as a collection of stopwords

In [4]:
##### The first 45 lines of the CACM dataset forms the first record
# We are interested only in 3 fields. 
# 1. the '.I' field, which is the document id
# 2. the '.T' field (the title) and
# 3. the '.W' field (the abstract, which may be absent)
with open ("./datasets/cacm.all","r") as file:
    cacm_all = "".join(file.readlines()[:45])
    print(cacm_all)
#####

.I 1
.T
Preliminary Report-International Algebraic Language
.B
CACM December, 1958
.A
Perlis, A. J.
Samelson,K.
.N
CA581203 JB March 22, 1978  8:28 PM
.X
100	5	1
123	5	1
164	5	1
1	5	1
1	5	1
1	5	1
205	5	1
210	5	1
214	5	1
1982	5	1
398	5	1
642	5	1
669	5	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
165	6	1
196	6	1
196	6	1
1273	6	1
1883	6	1
324	6	1
43	6	1
53	6	1
91	6	1
410	6	1
3184	6	1



---

The following function reads the `cacm.all` file. Note that each document has a variable number of lines. The `.I` field denotes a new document:

In [5]:
def read_cacm_docs(root_folder = "./datasets/"):
    """
        Reads in the CACM documents. The dataset is assumed to be in the folder "./datasets/" by default
        Returns: A list of 2-tuples: (doc_id, document), where 'document' is a single string created by 
            appending the title and abstract (separated by a "\n"). 
            In case the record doesn't have an abstract, the document is composed only by the title
    """
    with open(os.path.join(root_folder, "cacm.all")) as reader:
        lines = reader.readlines()
    
    doc_id, title, abstract = None, None, None
    
    docs = []
    line_idx = 0
    while line_idx < len(lines):
        line = lines[line_idx]
        if line.startswith(".I"):
            if doc_id is not None:
                docs.append((doc_id, title, abstract))
                doc_id, title, abstract = None, None, None
            
            doc_id = line.split()[-1]
            line_idx += 1
        elif line.startswith(".T"):
            # start at next line
            line_idx += 1
            temp_lines = []
            # read till next '.'
            while not lines[line_idx].startswith("."):
                temp_lines.append(lines[line_idx].strip("\n"))
                line_idx += 1
            title = "\n".join(temp_lines).strip("\n")
        elif line.startswith(".W"):
            # start at next line
            line_idx += 1
            temp_lines = []
            # read till next '.'
            while not lines[line_idx].startswith("."):
                temp_lines.append(lines[line_idx].strip("\n"))
                line_idx += 1
            abstract = "\n".join(temp_lines).strip("\n")
        else:
            line_idx += 1
    
    docs.append((doc_id, title, abstract))
    
    p_docs = []
    for (did, t, a) in docs:
        if a is None:
            a = ""
        p_docs.append((did, t + "\n" + a))
    return p_docs


In [6]:
##### Function check
docs = read_cacm_docs()

assert isinstance(docs, list)
assert len(docs) == 3204, "There should be exactly 3204 documents"

unzipped_docs = list(zip(*docs))
assert np.sum(np.array(list(map(int,unzipped_docs[0])))) == 5134410

##### 

---
### 1.2 Read the CACM queries

Next, let us read the queries. They are formatted similarly:

In [7]:
##### The first 15 lines of 'query.text' has 2 queries
# We are interested only in 2 fields. 
# 1. the '.I' - the query id
# 2. the '.W' - the query
!head -15 ./datasets/query.text
#####

'head' is not recognized as an internal or external command,
operable program or batch file.


---

The following function reads the `query.text` file:

In [8]:
def read_queries(root_folder = "./datasets/"):
    """
        Reads in the CACM queries. The dataset is assumed to be in the folder "./datasets/" by default
        Returns: A list of 2-tuples: (query_id, query)
    """
    with open(os.path.join(root_folder, "query.text")) as reader:
        lines = reader.readlines()
    
    query_id, query = None, None
    
    queries = []
    line_idx = 0
    while line_idx < len(lines):
        line = lines[line_idx]
        if line.startswith(".I"):
            if query_id is not None:
                queries.append((query_id, query))
                query_id, query = None, None
    
            query_id = line.split()[-1]
            line_idx += 1
        elif line.startswith(".W"):
            # start at next line
            line_idx += 1
            temp_lines = []
            # read till next '.'
            while not lines[line_idx].startswith("."):
                temp_lines.append(lines[line_idx].strip("\n"))
                line_idx += 1
            query = "\n".join(temp_lines).strip("\n")
        else:
            line_idx += 1
    
    queries.append((query_id, query))
    return queries


In [9]:
##### Function check
queries = read_queries()

assert isinstance(queries, list)
assert len(queries) == 64 and all([q[1] is not None for q in queries]), "There should be exactly 64 queries"

unzipped_queries = list(zip(*queries))
assert np.sum(np.array(list(map(int,unzipped_queries[0])))) == 2080

##### 

---
### 1.3 Read the stop words

We use the common words stored in `common_words`:

In [10]:
##### Read the stop words file 
!head ./datasets/common_words
##### Read the README file 

'head' is not recognized as an internal or external command,
operable program or batch file.


---

The following function reads the `common_words` file (For better coverage, we try to keep them in lowercase):

In [11]:
def load_stopwords(root_folder = "./datasets/"):
    """
        Loads the stopwords. The dataset is assumed to be in the folder "./datasets/" by default
        Output: A set of stopwords
    """
    with open(os.path.join(root_folder, "common_words")) as reader:
        lines = reader.readlines()
    stopwords = set([l.strip().lower() for l in lines])
    return stopwords


In [12]:
##### Function check
stopwords = load_stopwords()

assert isinstance(stopwords, set)
assert len(stopwords) == 428, "There should be exactly 428 stop words"

assert np.sum(np.array(list(map(len,stopwords)))) == 2234

##### 


---
### 1.4 Tokenization (3 points)

We can now write some basic text processing functions. 
A first step is to tokenize the text. 

**Note**: Use the  `WordPunctTokenizer` available in the `nltk` library:

In [13]:
# TODO: Implement this! (4 points)
def tokenize(text):
    """
        Tokenizes the input text. Use the WordPunctTokenizer
        Input: text - a string
        Output: a list of tokens
    """
    tk = nltk.tokenize.WordPunctTokenizer()
    return tk.tokenize(text)

In [14]:
##### Function check
text = "the quick brown fox jumps over the lazy dog"
tokens = tokenize(text)

assert isinstance(tokens, list)
assert len(tokens) == 9

print(tokens)
# output: ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
#####

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']


---
### 1.5 Stemming (2 points)

Write a function to stem tokens. 
Again, you can use the nltk library for this:

In [15]:
# TODO: Implement this! (3 points)
def stem_token(token):
    """
        Stems the given token using the PorterStemmer from the nltk library
        Input: a single token
        Output: the stem of the token
    """
    stemmer=nltk.stem.porter.PorterStemmer()
    return stemmer.stem(token)

In [16]:
##### Function check

assert stem_token('owned') == 'own'
assert stem_token('itemization') == 'item'
#####

---
### 1.6 Summary

The following function puts it all together. Given an input string, this functions tokenizes and processes it according to the flags that you set.

In [17]:
#### Putting it all together
def process_text(text, stem=False, remove_stopwords=False, lowercase_text=False):
    
    tokens = []
    for token in tokenize(text):
        if remove_stopwords and token.lower() in stopwords:
            continue
        if stem:
            token = stem_token(token)
        if lowercase_text:
            token = token.lower()
        tokens.append(token)

    return tokens
#### 

---

Let's create two sets of preprocessed documents.
We can process the documents and queries according to these two configurations:

In [18]:
# In this configuration:
# Don't preprocess the text, except to tokenize 
config_1 = {
  "stem": False,
  "remove_stopwords" : False,
  "lowercase_text": True
} 


# In this configuration:
# Preprocess the text, stem and remove stopwords
config_2 = {
  "stem": True,
  "remove_stopwords" : True,
  "lowercase_text": True, 
} 

####
doc_repr_1 = []
doc_repr_2 = []
for (doc_id, document) in docs:
    doc_repr_1.append((doc_id, process_text(document, **config_1)))
    doc_repr_2.append((doc_id, process_text(document, **config_2)))

####

--- 

## Section 2: Indexing (10 points)<a class="anchor" id="indexing"></a>

[Back to Part 1](#part1)



A retrieval function usually takes in a query document pair, and scores a query against a document.  Our document set is quite small - just a few thousand documents. However, consider a web-scale dataset with a few million documents. In such a scenario, it would become infeasible to score every query and document pair. In such a case, we can build an inverted index. From Wikipedia:

> ... , an inverted index (also referred to as a postings file or inverted file) is a database index storing a mapping from content, such as words or numbers, to its locations in a table, .... The purpose of an inverted index is to allow fast full-text searches, at a cost of increased processing when a document is added to the database. ...


Consider a simple inverted index, which maps from word to document. This can improve the performance of a retrieval system significantly. In this assignment, we consider a *simple* inverted index, which maps a word to a set of documents. In practice, however, more complex indices might be used.  


### 2.1 Term Frequency-index (10 points)
In this assignment, we will be using an index created in memory since our dataset is tiny. To get started, build a simple index that maps each `token` to a list of `(doc_id, count)` where `count` is the count of the `token` in `doc_id`.
For consistency, build this index using a python dictionary.
    
Now, implement a function to build an index:

In [240]:
# TODO: Implement this! (10 points)
from collections import defaultdict
def build_tf_index(documents):
    """
        Build an inverted index (with counts). The output is a dictionary which takes in a token
        and returns a list of (doc_id, count) where 'count' is the count of the 'token' in 'doc_id'
        Input: a list of documents - (doc_id, tokens) 
        Output: An inverted index implemented within a pyhton dictionary: [token] -> [(doc_id, token_count)]
    """
    # YOUR CODE HERE
    index = defaultdict(list)
    for (doc_id, tokens) in documents:
        doc_id = int(doc_id)
        seen_tks = {}
        for tk in tokens:
            new_tk = tk not in index
            if new_tk:
                index[tk] = []
            if tk not in seen_tks:
                index[tk].append([doc_id, 1])
                seen_tks[tk] = True
            else:
                index[tk][-1][1] += 1
    return index

---
Now we can build indexed documents and preprocess the queries based on the two configurations:

In [241]:
#### Indexed documents based on the two configs

# Create the 2 indices
tf_index_1 = build_tf_index(doc_repr_1)
tf_index_2 = build_tf_index(doc_repr_2)

# This function returns the tf_index of the corresponding config
def get_index(index_set):
    assert index_set in {1, 2}
    return {
        1: tf_index_1,
        2: tf_index_2
    }[index_set]

####
#### Preprocessed query based on the two configs

# This function preprocesses the text given the index set, according to the specified config
def preprocess_query(text, index_set):
    assert index_set in {1, 2}
    if index_set == 1:
        return process_text(text, **config_1)
    elif index_set == 2:
        return process_text(text, **config_2)

#### 

In [ ]:
##### Function check

assert isinstance(tf_index_1, dict)

assert isinstance(tf_index_1['computer'], list)
print('sample tf index for computer:', tf_index_1['computer'][:10])

assert isinstance(tf_index_1['examples'], list)
print('sample tf index for examples:', tf_index_1['examples'][:10])
#### 

In [ ]:
##### Function check

assert isinstance(tf_index_2, dict)

assert isinstance(tf_index_2['computer'], list)
print('sample tf index for computer:', tf_index_1['computer'][:10])

assert isinstance(tf_index_2['examples'], list)
print('sample tf index for examples:', tf_index_2['examples'][:10])
#### 


---
## Section 3: Ranking  (80 points) <a class="anchor" id="ranking"></a>

[Back to Part 1](#part1)

Now that we have cleaned and processed our dataset, we can start building simple IR systems. 

For now, we consider *simple* IR systems, which involve computing scores from the tokens present in the document/query. More advanced methods are covered in later assignments.

We will implement the following methods in this section:
- [Section 3.1: Bag of Words](#bow) (10 points)
- [Section 3.2: TF-IDF](#tfidf) (15 points)
- [Section 3.3: Query Likelihood Model](#qlm) (35 points)
- [Section 3.4: BM25](#bm25) (20 points)

*All search functions should be able to handle multiple words queries.*

**Scoring policy:**
Your implementations in this section are scored based on the expected performance of your ranking functions.
You will get a full mark if your implementation meets the expected performance (measured by some evaluation metric).
Otherwise, you may get partial credit.
For example, if your *Bag of words* ranking function has 60% of expected performance, you will get 6 out of 10.

--- 

### Section 3.1: Bag of Words (10 points)<a class="anchor" id="bow"></a>

Probably the simplest IR model is the Bag of Words (BOW) model.
Implement a function that scores and ranks all the documents against a query using this model.   

- For consistency, you should use the count of the token and **not** the binary indicator.
- Use `float` type for the scores (even though the scores are integers in this case).
- No normalization of the scores is necessary, as the ordering is what we are interested in.
- If two documents have the same score, they can have any ordering: you are not required to disambiguate.


In [252]:
# TODO: Implement this! (10 points)
def dict_to_arr(d, descending=True):
    arr = []
    for k in d.keys():
         arr.append([str(k), d[k]])
    return sorted(arr, key=lambda x: x[1], reverse=descending)

def bow_search(query, index_set):
    """
        Perform a search over all documents with the given query. 
        Note: You have to use the `get_index` function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query. 
    """
    index = get_index(index_set)
    processed_query = preprocess_query(query, index_set)
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
#### Function check

test_bow = bow_search("how to implement bag of words search", index_set=1)[:5]
assert isinstance(test_bow, list)
assert len(test_bow[0]) == 2
assert isinstance(test_bow[0][0], str)
assert isinstance(test_bow[0][1], float)

#### 

In [ ]:

docs_by_id = dict(docs)
def print_results(docs, len_limit=50):    
    for i, (doc_id, score) in enumerate(docs):
        doc_content = docs_by_id[doc_id].strip().replace("\n", "\\n")[:len_limit] + "..."
        print(f"Rank {i}({score:.2}): {doc_content}")

test_bow_2 = bow_search("computer search word", index_set=2)[:5]
print(f"BOW Results:")
print_results(test_bow_2)


In [ ]:

test_bow_1 = bow_search("computer search word", index_set=1)[:5]
print(f"BOW Results:")
print_results(test_bow_1)


In [ ]:
print('top-5 docs for index1:', list(zip(*test_bow_1[:5]))[0])
print('top-5 docs for index2:', list(zip(*test_bow_2[:5]))[0])



---

### Section 3.2: TF-IDF (15 points) <a class="anchor" id="tfidf"></a>

Before we implement the tf-idf scoring functions, let's first write a function to compute the document frequencies of all words.  

#### 3.2.1 Document frequency (5 points)
Compute the document frequencies of all tokens in the collection. 
Your code should return a dictionary with tokens as its keys and the number of documents containing the token as values.
For consistency, the values should have `int` type.

In [245]:
# TODO: Implement this! (5 points)
def compute_df(documents):
    """
        Compute the document frequency of all terms in the vocabulary
        Input: A list of documents
        Output: A dictionary with {token: document frequency (int)}
    """
    # YOUR CODE HERE
    df = {}
    for doc in documents:
        seen_tks = {}
        for tk in doc:
            if tk not in df:
                df[tk] = 0
            df[tk] += not tk in seen_tks
            seen_tks[tk] = True
    return df

In [246]:
#### Compute df based on the two configs

# get the document frequencies of each document
df_1 = compute_df([d[1] for d in doc_repr_1])
df_2 = compute_df([d[1] for d in doc_repr_2])

def get_df(index_set):
    assert index_set in {1, 2}
    return {
        1: df_1,
        2: df_2
    }[index_set]
####

In [ ]:
#### Function check

print(df_1['computer'])
print(df_2['computer'])
####

---
#### 3.2.2 TF-IDF search (10 points)
Next, implement a function that computes a tf-idf score, given a query.
Use the following formulas for TF and IDF:

$$ TF=\log (1 + f_{d,t}) $$

$$ IDF=\log (\frac{N}{n_t})$$

where $f_{d,t}$ is the frequency of token $t$ in document $d$, $N$ is the number of total documents and $n_t$ is the number of documents containing token $t$.

**Note:** your implementation will be auto-graded assuming you have used the above formulas.


In [ ]:
# TODO: Implement this! (10 points)
def tfidf_search(query, index_set):
    """
        Perform a search over all documents with the given query using tf-idf. 
        Note #1: You have to use the `get_index` (and the `get_df`) function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    df = get_df(index_set)
    processed_query = preprocess_query(query, index_set)
    
    N = len(docs)
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:

#### Function check
test_tfidf = tfidf_search("how to implement tf idf search", index_set=1)[:5]
assert isinstance(test_tfidf, list)
assert len(test_tfidf[0]) == 2
assert isinstance(test_tfidf[0][0], str)
assert isinstance(test_tfidf[0][1], float)

####

In [ ]:

test_tfidf_2 = tfidf_search("computer word search", index_set=2)[:5]
print(f"TFIDF Results:")
print_results(test_tfidf_2)


In [ ]:

test_tfidf_1 = tfidf_search("computer word search", index_set=1)[:5]
print(f"TFIDF Results:")
print_results(test_tfidf_1)


In [ ]:

print('top-5 docs for index1 with BOW search:', list(zip(*test_bow_1[:5]))[0])
print('top-5 docs for index2 with BOW search:', list(zip(*test_bow_2[:5]))[0])
print('top-5 docs for index1 with TF-IDF search:', list(zip(*test_tfidf_1[:5]))[0])
print('top-5 docs for index2 with TF-IDF search:', list(zip(*test_tfidf_2[:5]))[0])



--- 

### Section 3.3: Query Likelihood Model (35 points) <a class="anchor" id="qlm"></a>

In this section, you will implement a simple query likelihood model. 


#### 3.3.1 Naive QL (15 points)

First, let us implement a naive version of a QL model, assuming a multinomial unigram language model (with a uniform prior over the documents). 



In [249]:
#### Document length for normalization

def doc_lengths(documents):
    doc_lengths = {doc_id:len(doc) for (doc_id, doc) in documents}
    return doc_lengths

doc_lengths_1 = doc_lengths(doc_repr_1)
doc_lengths_2 = doc_lengths(doc_repr_2)

def get_doc_lengths(index_set):
    assert index_set in {1, 2}
    return {
        1: doc_lengths_1,
        2: doc_lengths_2
    }[index_set]
####

In [ ]:
# TODO: Implement this! (15 points)
def naive_ql_search(query, index_set):
    """
        Perform a search over all documents with the given query using a naive QL model. 
        Note #1: You have to use the `get_index` (and get_doc_lengths) function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    doc_lengths = get_doc_lengths(index_set)
    processed_query = preprocess_query(query, index_set)
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
#### Function check
test_naiveql = naive_ql_search("report", index_set=1)[:5]
print(f"Naive QL Results:")
print_results(test_naiveql)
####

In [ ]:
#### Please do not change this. This cell is used for grading.

In [ ]:
#### Please do not change this. This cell is used for grading.

In [ ]:
#### Please do not change this. This cell is used for grading.

In [ ]:
#### Please do not change this. This cell is used for grading.

---
#### 3.3.2 QL (20 points)
Now, let's implement a QL model that handles the issues with the naive version. In particular, you will implement a QL model with Jelinek-Mercer Smoothing. That means an interpolated score is computed per word - one term is the same as the previous naive version, and the second term comes from a unigram language model. In addition, you should accumulate the scores by summing the **log** (smoothed) probability which leads to better numerical stability.

In [ ]:
# TODO: Implement this! (20 points)

# YOUR CODE HERE
raise NotImplementedError()

def ql_search(query, index_set):
    """
        Perform a search over all documents with the given query using a QL model 
        with Jelinek-Mercer Smoothing (set smoothing=0.1). 
        
        
        Note #1: You have to use the `get_index` (and get_doc_lengths) function created in the previous cells
        Note #2: You might have to create some variables beforehand and use them in this function
        
        
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    doc_lengths = get_doc_lengths(index_set)
    processed_query = preprocess_query(query, index_set)
    
    # YOUR CODE HERE
    raise NotImplementedError()
    

In [ ]:
#### Function check
test_ql_results = ql_search("report", index_set=1)[:5]
print_results(test_ql_results)
print()
test_ql_results_long = ql_search("report " * 10, index_set=1)[:5]
print_results(test_ql_results_long)
####

In [ ]:
#### Please do not change this. This cell is used for grading.

In [ ]:
#### Please do not change this. This cell is used for grading.

In [ ]:
#### Please do not change this. This cell is used for grading.

In [ ]:
#### Please do not change this. This cell is used for grading.

--- 

### Section 3.4: BM25 (20 points) <a class="anchor" id="bm25"></a>

In this section, we will implement the BM25 scoring function. 


In [253]:
# TODO: Implement this! (20 points)
def bm25_search(query, index_set):
    """
        Perform a search over all documents with the given query using BM25. Use k_1 = 1.5 and b = 0.75
        Note #1: You have to use the `get_index` (and `get_doc_lengths`) function created in the previous cells
        Note #2: You might have to create some variables beforehand and use them in this function
        
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    k_1 = 1.5
    b = 0.75
    index = get_index(index_set)
    df = get_df(index_set)
    doc_lengths = get_doc_lengths(index_set)
    processed_query = preprocess_query(query, index_set)
    scores = {}
    vals = doc_lengths.values()
    N = len(vals)
    dl_avg = sum(vals)/len(vals)
    for q in processed_query:
        if q not in index: continue
        for (doc_id, freq) in index[q]:
            if doc_id not in scores:
                scores[doc_id] = 0
            scores[doc_id] += np.log(N/df[q])*(k_1+1)*freq/(k_1*(1-b + b*doc_lengths[str(doc_id)]/dl_avg)+freq)
    return dict_to_arr(scores)

In [254]:
#### Function check
test_bm25_results = bm25_search("report", index_set=1)[:5]
print_results(test_bm25_results)
####

NameError: name 'print_results' is not defined

In [ ]:
#### Please do not change this. This cell is used for grading.

In [ ]:
#### Please do not change this. This cell is used for grading.

In [ ]:
#### Please do not change this. This cell is used for grading.

In [ ]:
#### Please do not change this. This cell is used for grading.


---

### 3.5. Test Your Functions

The widget below allows you to play with the search functions you've written so far. Use this to test your search functions and ensure that they work as expected.

In [30]:
#### Highlighter function
# class for results
ResultRow = namedtuple("ResultRow", ["doc_id", "snippet", "score"])
# doc_id -> doc
docs_by_id = dict((d[0], d[1]) for d in docs)

def highlight_text(document, query, tol=17):
    import re
    tokens = tokenize(query)
    regex = "|".join(f"(\\b{t}\\b)" for t in tokens)
    regex = re.compile(regex, flags=re.IGNORECASE)
    output = ""
    i = 0
    for m in regex.finditer(document):
        start_idx = max(0, m.start() - tol)
        end_idx = min(len(document), m.end() + tol)
        output += "".join(["...",
                        document[start_idx:m.start()],
                        "<strong>",
                        document[m.start():m.end()],
                        "</strong>",
                        document[m.end():end_idx],
                        "..."])
    return output.replace("\n", " ")


def make_results(query, search_fn, index_set):
    results = []
    for doc_id, score in search_fn(query, index_set):
        highlight = highlight_text(docs_by_id[doc_id], query)
        if len(highlight.strip()) == 0:
            highlight = docs_by_id[doc_id]
        results.append(ResultRow(doc_id, highlight, score))
    return results
####

In [84]:
# TODO: Set this to the function you want to test!
# this function should take in a query (string)
# and return a sorted list of (doc_id, score) 
# with the most relevant document in the first position
search_fn = bm25_search
index_set = 1

text = widgets.Text(description="Search Bar", width=200)
display(text)

def handle_submit(sender):
    print(f"Searching for: '{sender.value}'")
    
    results = make_results(sender.value, search_fn, index_set)
    
    # display only the top 5
    results = results[:5]
    
    body = ""
    for idx, r in enumerate(results):
        body += f"<li>Document #{r.doc_id}({r.score}): {r.snippet}</li>"
    display(HTML(f"<ul>{body}</ul>"))
    

text.on_submit(handle_submit)

NameError: name 'bm25_search' is not defined

---

## Section 4: Evaluation (40 points) <a class="anchor" id="evaluation"></a>

[Back to Part 1](#part1)

In order to analyze the effectiveness of retrieval algorithms, we first have to learn how to evaluate such a system. In particular, we will work with offline evaluation metrics. These metrics are computed on a dataset with known relevance judgements.

Implement the following evaluation metrics. 

1. Precision (7 points)
2. Recall (7 points)
3. Mean Average Precision (13 points)
4. Expected Reciprocal Rank (13 points)

---
### 4.1 Read relevance labels

Let's take a look at the `qrels.text` file, which contains the ground truth relevance scores. The relevance labels for CACM are binary - either 0 or 1. 


In [ ]:
!head ./datasets/qrels.text

---

The first column is the query_id and the second column is the document_id. We can safely ignore the 3rd and 4th columns.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#### Function check
qrels = read_qrels()

assert len(qrels) == 52, "There should be 52 queries with relevance judgements"
assert sum(len(j) for j in qrels.values()) == 796, "There should be a total of 796 Relevance Judgements"

assert np.min(np.array([len(j) for j in qrels.values()])) == 1
assert np.max(np.array([len(j) for j in qrels.values()])) == 51

####

---
**Note:** For a given query `query_id`, you can assume that documents *not* in `qrels[query_id]` are not relevant to `query_id`. 


---
### 4.2 Precision (7 points)
Implement the `precision@k` metric:

In [ ]:
# TODO: Implement this! (7 points)
def precision_k(results, relevant_docs, k):
    """
        Compute Precision@K
        Input: 
            results: A sorted list of 2-tuples (document_id, score), 
                    with the most relevant document in the first position
            relevant_docs: A set of relevant documents. 
            k: the cut-off
        Output: Precision@K
    """
    if k > len(results):
        k = len(results)
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:

#### Function check
qid = queries[0][0]
qtext = queries[0][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
precision = precision_k(results, qrels[qid], 10)
print(f'precision@10 = {precision}')
####

---
### 4.3 Recall (7 points)
Implement the `recall@k` metric:

In [ ]:
# TODO: Implement this! (7 points)
def recall_k(results, relevant_docs, k):
    """
        Compute Recall@K
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most relevant document in the first position
            relevant_docs: A set of relevant documents. 
            k: the cut-off
        Output: Recall@K
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
#### Function check
qid = queries[10][0]
qtext = queries[10][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
recall = recall_k(results, qrels[qid], 10)
print(f'recall@10 = {recall}')
####

---
### 4.4 Mean Average Precision (13 points)
Implement the `map` metric:

In [ ]:
# TODO: Implement this! (12 points)
def average_precision(results, relevant_docs):
    """
        Compute Average Precision (for a single query - the results are 
        averaged across queries to get MAP in the next few cells)
        Hint: You can use the recall_k and precision_k functions here!
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most 
                    relevant document in the first position
            relevant_docs: A set of relevant documents. 
        Output: Average Precision
    """
    # YOUR CODE HERE
    raise NotImplementedError()


In [ ]:
#### Function check
qid = queries[20][0]
qtext = queries[20][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
mean_ap = average_precision(results, qrels[qid])
print(f'MAP = {mean_ap}')
####

---
### 4.5 Expected Reciprocal Rank (13 points)
Implement the `err` metric:

In [ ]:
# TODO: Implement this! (12 points)
def err(results, relevant_docs):
    """
        Compute the expected reciprocal rank.
        Hint: https://dl.acm.org/doi/pdf/10.1145/1645953.1646033?download=true
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most 
                    relevant document in the first position
            relevant_docs: A set of relevant documents. 
        Output: ERR
        
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
#### Function check
qid = queries[30][0]
qtext = queries[30][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
ERR = err(results, qrels[qid])
print(f'ERR = {ERR}')
####

---
### 4.6 Evaluate Search Functions

Let's define some metrics@k using [partial functions](https://docs.python.org/3/library/functools.html#functools.partial)

In [ ]:
#### metrics@k functions

recall_at_1 = partial(recall_k, k=1)
recall_at_5 = partial(recall_k, k=5)
recall_at_10 = partial(recall_k, k=10)
precision_at_1 = partial(precision_k, k=1)
precision_at_5 = partial(precision_k, k=5)
precision_at_10 = partial(precision_k, k=10)


list_of_metrics = [
    ("ERR", err),
    ("MAP", average_precision),
    ("Recall@1",recall_at_1),
    ("Recall@5", recall_at_5),
    ("Recall@10", recall_at_10),
    ("Precision@1", precision_at_1),
    ("Precision@5", precision_at_5),
    ("Precision@10", precision_at_10)]
####

---

The following function evaluates a `search_fn` using the `metric_fn`. Note that the final number is averaged over all the queries

In [ ]:
#### Evaluate a search function

list_of_search_fns = [
    ("BOW", bow_search),
    ("TF-IDF", tfidf_search),
    ("NaiveQL", naive_ql_search),
    ("QL", ql_search),
    ("BM25", bm25_search)
]

def evaluate_search_fn(search_fn, metric_fns, index_set=None):
    # build a dict query_id -> query 
    queries_by_id = dict((q[0], q[1]) for q in queries)
    
    metrics = {}
    for metric, metric_fn in metric_fns:
        metrics[metric] = np.zeros(len(qrels), dtype=np.float32)
    
    for i, (query_id, relevant_docs) in enumerate(qrels.items()):
        query = queries_by_id[query_id]
        if index_set:
            results = search_fn(query, index_set)
        else:
            results = search_fn(query)
        
        for metric, metric_fn in metric_fns:
            metrics[metric][i] = metric_fn(results, relevant_docs)

    
    
    final_dict = {}
    for metric, metric_vals in metrics.items():
        final_dict[metric] = metric_vals.mean()
    
    return final_dict
####

## Section 5: Analysis (30 points) <a class="anchor" id="analysis"></a>

[Back to Part 1](#part1)

In the final section of Part1, we will compare the different term-based IR algorithms and different preprocessing configurations and analyze their advantages and disadvantages.

### Section 5.1: Plot (20 points)

First, gather the results. The results should consider the index set, the different search functions and different metrics. Plot the results in bar charts, per metric, with clear labels.

**Rubric:**
- Each Metric is plotted: 7 points
- Each Method is plotted: 7 points
- Clear titles, x label, y labels and legends (if applicable): 6 points

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

---
### Section 5.2: Summary (10 points)
Write a summary of what you observe in the results.
Your summary should compare results across the 2 indices and the methods being used. State what you expected to see in the results, followed by either supporting evidence *or* justify why the results did not support your expectations.      

Write your answer here!

---
---
# Part 2: Semantic-based Matching (85 points) <a class="anchor" id="part2"></a>

[Back to top](#top)

We will now experiment with methods that go beyond lexical methods like TF-IDF, which operate at the word level and are high dimensional and sparse, and look at methods which constructs low dimensional dense representations of queries and documents. 

Since these low-dimensional methods have a higher time complexity, they are typically used in conjunction with methods like BM-25. That is, instead of searching through potentially million documents to find matches using low dimensional vectors, a list of K documents are retrieved using BM25, and then **re-ranked** using the other method. This is the method that is going to be applied in the following exercises. 

LSI/LDA takes documents that are similar on a semantic level - for instance, if they are describing the same topic - and projects them into nearby vectors, despite having low lexical overlap.

In this assignment, you will use `gensim` to create LSI/LDA models and use them in re-ranking. 

**Note**: The following exercises only uses `doc_repr_2` and `config_2`

Table of contents:
- [Section 6: LSI](#lsi) (15 points)
- [Section 7: LDA](#lda) (10 points)
- [Section 8: Word2Vec/Doc2Vec](#2vec) (20 points)
- [Section 8: Re-ranking](#reranking) (10 points)
- [Section 9: Re-ranking Evaluation](#reranking_eval) (30 points)

---
## Section 6: Latent Semantic Indexing (LSI) (15 points) <a class="anchor" id="lsi"></a>

[Back to Part 2](#part2)

LSI is one of the methods to embed the queries and documents into vectors. It is based on a method similar to Principal Component Analysis (PCA) for obtaining a dense concept matrix out of the sparse term-document matrix.

See [wikipedia](https://en.wikipedia.org/wiki/Latent_semantic_analysis), particularly [#Mathematics_of_LSI](https://en.wikipedia.org/wiki/Latent_semantic_analysis#Mathematics_of_LSI).

In [19]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel, LsiModel, Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import downloader as g_downloader
# gensim uses logging, so set it up 
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

---
### Section 6.1: Cosine Similarity (5 points)<a class="anchor" id="cosing_sim"></a>
Before we begin, let us first define our method of similarity for the LSI model, the cosine similarity:

$$\text{similarity} = \cos(\theta) = {\mathbf{A} \cdot \mathbf{B} \over \|\mathbf{A}\| \|\mathbf{B}\|} = \frac{ \sum\limits_{i=1}^{n}{A_i  B_i} }{ \sqrt{\sum\limits_{i=1}^{n}{A_i^2}}  \sqrt{\sum\limits_{i=1}^{n}{B_i^2}} }$$

Since we are using gensim, the types of vectors returned by their classes are of the form defined below (they are not just simple vectors):

In [20]:
# 1, 2, 3 are either latent dimensions (LSI), or topics (LDA)
# The second value in each tuple is a number (LSI) or a probability (LDA)  
example_vec_1 = [(1, 0.2), (2, 0.3), (3, 0.4)]
example_vec_2 = [(1, 0.2), (2, 0.7), (3, 0.4)]

---
**Implementation (2+3 points):**
Now, implement the `dot product` operation on these types of vectors and using this operator, implement the `cosine similarity` (don't forget: two functions to implement!):

In [21]:
def dot(vec_1,vec_2): 
    """
        vec_1 and vec_2 are of the form: [(int, float), (int, float), ...]
        Return the dot product of two such vectors, computed only on the floats
        You can assume that the lengths of the vectors are the same, and the dimensions are aligned 
            i.e you won't get: vec_1 = [(1, 0.2)] ; vec_2 = [(2, 0.3)] 
                                (dimensions are unaligned and lengths are different)
    """
    ar_1=np.array(vec_1)[:,1]
    ar_2=np.array(vec_2)[:,1]
    combined=np.dot(ar_1,ar_2)
    return combined


# TODO: Implement this! (3 points)
def cosine_sim(vec_1, vec_2):
    d = dot(vec_1,vec_2)
    ar_1=np.array(vec_1)[:,1]
    ar_2=np.array(vec_2)[:,1]
    squar_1=np.sqrt(np.sum(np.square(ar_1)))
    squar_2=np.sqrt(np.sum(np.square(ar_2)))
    
    return d/(np.dot(squar_1,squar_2))

In [22]:
##### Function check
print(f'vectors: {(example_vec_1,example_vec_2)}')
print(f'dot product = {dot(example_vec_1,example_vec_2)}')
print(f'cosine similarity = {cosine_sim(example_vec_1,example_vec_2)}')
##### 

vectors: ([(1, 0.2), (2, 0.3), (3, 0.4)], [(1, 0.2), (2, 0.7), (3, 0.4)])
dot product = 0.41000000000000003
cosine similarity = 0.9165587597202866


In [23]:
#### Please do not change this. This cell is used for grading.

---
### Section 6.2: LSI Retrieval (10 points)<a class="anchor" id="lsi_retrieval"></a>
LSI retrieval is simply ranking the documents based on their cosine similarity to the query vector.
First, let's write a parent class for vector-based retrieval models:

In [24]:
class VectorSpaceRetrievalModel:
    """
        Parent class for Dense Vector Retrieval models
    """
    def __init__(self, doc_repr):
        """
            document_collection: 
                [
                    (doc_id_1, [token 1, token 2, ...]), 
                    (doc_id_2, [token 1, token 2, ....]) 
                    ...
                ]

        """
        self.doc_repr = doc_repr
        self.documents = [_[1] for _ in self.doc_repr]
        
        # construct a dictionary
        self.dictionary = Dictionary(self.documents)
        # Filter out words that occur less than 20 documents, or more than 50% of the documents.
        self.dictionary.filter_extremes(no_below=10)
        self.corpus = [self.dictionary.doc2bow(doc) for doc in self.documents]
    
        # Make a index to word dictionary.
        temp = self.dictionary[0]  # This is only to "load" the dictionary.
        self.id2word = self.dictionary.id2token
        
        # this is set by the train_model function
        self.model = None
        
        
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """
        vectors = {}
        for (doc_id, _), cc in zip(self.doc_repr, self.corpus):
            vectors[doc_id] = self.model[cc]
        return vectors

    def vectorize_query(self, query):
        # Note the use of config_2 here!
        query = process_text(query, **config_2)
        query_vector = self.dictionary.doc2bow(query)
        return self.model[query_vector]
    
    def train_model(self):
        """
            Trains a model and sets the 'self.model' variable. 
            Make sure to use the variables created in the __init__ method.
            e.g the variables which may be useful: {corpus, dictionary, id2word}
        """
        raise NotImplementedError()

---
**Implementation (5 points):**
Implement the `train_model` method in the following class (note that this is only one line of code in `gensim`!). Ensure that the parameters defined in the `__init__` method are not changed, and are *used in the `train_method` function*. Normally, the hyperaparameter space will be searched using grid search / other methods - in this assignment we have provided the hyperparameters for you.

The last two lines of code train an LSI model on the list of documents which have been stemmed, lower-cased and have stopwords removed. 

In [25]:
# TODO: Implement this! (5 points)
class LsiRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        self.num_topics = 100
        self.chunksize = 2000
    
    def train_model(self):
        # YOUR CODE HERE
        self.model=LsiModel(self.corpus, id2word=self.id2word, num_topics=self.num_topics, chunksize=self.chunksize)

In [26]:
##### Function check
lsi = LsiRetrievalModel(doc_repr_2)
lsi.train_model()

# you can now get an LSI vector for a given query in the following way:
lsi.vectorize_query("report")
##### 

2022-02-24 11:15:14,485 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-02-24 11:15:14,586 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)
2022-02-24 11:15:14,587 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)", 'datetime': '2022-02-24T11:15:14.587449', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}
2022-02-24 11:15:14,593 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2022-02-24 11:15:14,593 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (

[(0, 0.015215631439977865),
 (1, -0.016278790162605035),
 (2, -0.0001541836565231193),
 (3, -0.0018059463951937917),
 (4, -0.009411474688430534),
 (5, -0.004803258836632836),
 (6, 0.02700537907444965),
 (7, 0.0165628229253435),
 (8, -0.03175437374174067),
 (9, -0.0008666954429372518),
 (10, 0.0020930476059984955),
 (11, -0.01746359701151132),
 (12, 0.0004108639238813751),
 (13, 0.001225869332146432),
 (14, 0.004370292306163186),
 (15, 0.00491933918189602),
 (16, 0.005255352472765701),
 (17, 0.0021872838388268152),
 (18, -0.017338662255069966),
 (19, 0.01977032810571728),
 (20, -0.01063992454896577),
 (21, -0.013266477413967276),
 (22, 0.049829503250589456),
 (23, 0.02399852877751616),
 (24, -0.011901649748994382),
 (25, -0.012132535726598268),
 (26, 0.006832857881685087),
 (27, 0.0748432230348226),
 (28, -0.06171480995711938),
 (29, 0.03383468489467766),
 (30, 0.04216589293963088),
 (31, 0.04895054815993258),
 (32, -0.07193370986641853),
 (33, 0.053454608029253343),
 (34, -0.0277706966

\#### Please do not change this. This cell is used for grading.

---
**Implementation (5 points):**
 Next, implement a basic ranking class for vector space retrieval (used for all semantic methods): 

In [516]:
# TODO: Implement this! (5 points)
class DenseRetrievalRanker:
    def __init__(self, vsrm, similarity_fn):
        """
            vsrm: instance of `VectorSpaceRetrievalModel`
            similarity_fn: function instance that takes in two vectors 
                            and returns a similarity score e.g cosine_sim defined earlier
        """
        self.vsrm = vsrm
        self.vectorized_documents = self.vsrm.vectorize_documents()
        self.similarity_fn = similarity_fn
    
    def _compute_sim(self, query_vector):
        """
            Compute the similarity of `query_vector` to documents in 
            `self.vectorized_documents` using `self.similarity_fn`
            Returns a list of (doc_id, score) tuples
        """
        tup=[]
        for i in self.vectorized_documents:
            
            if np.array(self.vectorized_documents[i]).size==0 or  np.array(query_vector).size==0:
                continue
            
            if np.array(self.vectorized_documents[i]).shape!=np.array(query_vector).shape:
                continue
            else:
                scor=self.similarity_fn(self.vectorized_documents[i],query_vector)
            tup.append((i,scor))
        return tup
    
    def search(self, query):
        scores = self._compute_sim(self.vsrm.vectorize_query(query))
        scores.sort(key=lambda _:-_[1])
        return scores 

In [517]:
##### Function check
drm_lsi = DenseRetrievalRanker(lsi, cosine_sim)
drm_lsi.search("report")[:5]
##### 

[('599', 0.7970838688136261),
 ('947', 0.5791857647410769),
 ('53', 0.4996494666951299),
 ('1339', 0.46564041830979813),
 ('3160', 0.4453413974273841)]

\#### Please do not change this. This cell is used for grading.

---
Now, you can test your LSI model in the following cell: try finding queries which are lexically different to documents, but semantically similar - does LSI work well for these queries?!

In [48]:
# test your LSI model
search_fn = drm_lsi.search

text = widgets.Text(description="Search Bar", width=200)
display(text)

def make_results_2(query, search_fn):
    results = []
    for doc_id, score in search_fn(query):
        highlight = highlight_text(docs_by_id[doc_id], query)
        if len(highlight.strip()) == 0:
            highlight = docs_by_id[doc_id]
        results.append(ResultRow(doc_id, highlight, score))
    return results

def handle_submit_2(sender):
    print(f"Searching for: '{sender.value}' (SEARCH FN: {search_fn})")
    
    results = make_results_2(sender.value, search_fn)
    
    # display only the top 5
    results = results[:5]
    
    body = ""
    for idx, r in enumerate(results):
        body += f"<li>Document #{r.doc_id}({r.score}): {r.snippet}</li>"
    display(HTML(f"<ul>{body}</ul>"))
    

text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

Searching for: 'naaad' (SEARCH FN: <bound method DenseRetrievalRanker.search of <__main__.DenseRetrievalRanker object at 0x00000270DF0C4550>>)


Searching for: 'badd' (SEARCH FN: <bound method DenseRetrievalRanker.search of <__main__.DenseRetrievalRanker object at 0x00000270DF0C4550>>)


Searching for: 'bad' (SEARCH FN: <bound method DenseRetrievalRanker.search of <__main__.DenseRetrievalRanker object at 0x00000270DF0C4550>>)


Searching for: 'probability' (SEARCH FN: <bound method DenseRetrievalRanker.search of <__main__.DenseRetrievalRanker object at 0x00000270DF0C4550>>)


---
## Section 7: Latent Dirichlet Allocation (LDA) (10 points) <a class="anchor" id="lda"></a>

[Back to Part 2](#part2)

The specifics of LDA is out of the scope of this assignment, but we will use the `gensim` implementation to perform search using LDA over our small document collection. The key thing to remember is that LDA, unlike LSI, outputs a topic **distribution**, not a vector. With that in mind, let's first define a similarity measure.


---
### Section 7.1: Jenson-Shannon divergence (5 points) <a class="anchor" id="js_sim"></a>

The Jenson-Shannon divergence is a symmetric and finite measure on two probability distributions (unlike the KL, which is neither). For identical distributions, the JSD is equal to 0, and since our code uses 0 as irrelevant and higher scores as relevant, we use `(1 - JSD)` as the score or 'similarity' in our setup

**Note**: the JSD is bounded to \[0,1\] only if we use log base 2. So please ensure that you're using `np.log2` instead of `np.log`

In [34]:
## TODO: Implement this! (5 points)
def KL_divergence(p, q):
    """ Compute KL divergence of two vectors, K(p || q)."""
    return sum(p[x] * np.log2((p[x]) / (q[x])) for x in range(len(p)) if p[x] != 0.0 or p[x] != 0)

def jenson_shannon_divergence(vec_1, vec_2, assert_prob=False):
    """
        Computes the Jensen-Shannon divergence between two probability distributions. 
        NOTE: DO NOT RETURN 1 - JSD here, that is handled by the next function which is already implemented! 
        The inputs are *gensim* vectors - same as the vectors for the cosine_sim function
        assert_prob is a flag that checks if the inputs are proper probability distributions 
            i.e they sum to 1 and are positive - use this to check your inputs if needed. 
                (This is optional to implement, but recommended - 
                you can the default to False to save a few ms off the runtime)
    """
    
    if not vec_1 or not vec_2:
        return 0
    vec_1=np.array(vec_1)[:,1]
    vec_2=np.array(vec_2)[:,1]
    if vec_1.size!=vec_2.size:
        return 0
    JSD = 0.0
    weight = 0.5
    average = np.zeros(len(vec_1)) #Average
    for x in range(len(vec_1)):
        average[x] = weight * vec_1[x] + (1 - weight) * vec_2[x]
        JSD = (weight * KL_divergence(np.array(vec_1), average)) + ((1 - weight) * KL_divergence(np.array(vec_2), average))
    return JSD/np.sqrt(2*0.5)
    

def jenson_shannon_sim(vec_1, vec_2, assert_prob=False):
    return 1 - jenson_shannon_divergence(vec_1, vec_2)



In [35]:
##### Function check
vec_1 = [(1, 0.3), (2, 0.4), (3, 0.3)]
vec_2 = [(1, 0.1), (2, 0.7), (3, 0.2)]
jenson_shannon_sim(vec_1, vec_2, assert_prob=True)
##### 

C:\Users\Bram\AppData\Local\Temp/ipykernel_15384/2120849064.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  return sum(p[x] * np.log2((p[x]) / (q[x])) for x in range(len(p)) if p[x] != 0.0 or p[x] != 0)


0.9251064410358459

---
### Section 7.2: LDA retrieval (5 points) <a class="anchor" id="lda_ret"></a>

Implement the `train_model` method in the following class (note that this is only one line of code in `gensim`!). Ensure that the parameters defined in the `__init__` method are not changed, and are *used in the `train_method` function*. You do not need to set this. Normally, the hyperaparameter space will be searched using grid search / other methods. Note that training the LDA model might take some time

The last two lines of code train an LDA model on the list of documents which have been stemmed, lower-cased and have stopwords removed. 

In [38]:
# TODO: Implement this! (5 points)
class LdaRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        # use these parameters in the train_model method
        self.num_topics = 100
        self.chunksize = 2000
        self.passes = 20
        self.iterations = 400
        self.eval_every = 10
        # this is need to get full vectors
        self.minimum_probability=0.0
        self.alpha='auto'
        self.eta='auto'
    
    
    def train_model(self):
        self.model=LdaModel(self.corpus, num_topics=self.num_topics, chunksize=self.chunksize, passes=self.passes, 
                           iterations=self.iterations, eval_every=self.eval_every, minimum_probability=self.minimum_probability,
                           alpha=self.alpha, eta=self.eta)

In [39]:
##### Function check
lda = LdaRetrievalModel(doc_repr_2)
lda.train_model()

# you can now get an LDA vector for a given query in the following way:
lda.vectorize_query("report")
##### 

2022-02-24 11:17:06,690 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-02-24 11:17:06,785 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)
2022-02-24 11:17:06,786 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)", 'datetime': '2022-02-24T11:17:06.786660', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}
2022-02-24 11:17:06,791 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2022-02-24 11:17:06,791 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (

2022-02-24 11:17:11,393 : INFO : merging changes from 2000 documents into a model of 3204 documents
2022-02-24 11:17:11,403 : INFO : topic #86 (0.010): 0.116*"398" + 0.097*"83" + 0.063*"202" + 0.057*"1091" + 0.052*"821" + 0.051*"193" + 0.044*"0" + 0.039*"194" + 0.036*"96" + 0.035*"703"
2022-02-24 11:17:11,404 : INFO : topic #82 (0.010): 0.184*"162" + 0.061*"368" + 0.056*"50" + 0.051*"15" + 0.044*"499" + 0.042*"903" + 0.035*"997" + 0.034*"520" + 0.029*"41" + 0.027*"404"
2022-02-24 11:17:11,404 : INFO : topic #43 (0.010): 0.075*"50" + 0.050*"49" + 0.027*"582" + 0.023*"13" + 0.020*"57" + 0.019*"603" + 0.018*"115" + 0.012*"3" + 0.012*"561" + 0.012*"45"
2022-02-24 11:17:11,405 : INFO : topic #40 (0.011): 0.104*"50" + 0.048*"49" + 0.033*"13" + 0.028*"0" + 0.017*"6" + 0.017*"239" + 0.012*"45" + 0.011*"166" + 0.010*"115" + 0.010*"57"
2022-02-24 11:17:11,405 : INFO : topic #77 (0.011): 0.191*"83" + 0.181*"34" + 0.104*"35" + 0.094*"611" + 0.072*"1015" + 0.026*"69" + 0.018*"404" + 0.015*"131" + 0

2022-02-24 11:17:15,550 : INFO : topic #91 (0.011): 0.237*"34" + 0.211*"35" + 0.163*"83" + 0.046*"254" + 0.035*"611" + 0.032*"1057" + 0.030*"210" + 0.026*"219" + 0.026*"1093" + 0.017*"1054"
2022-02-24 11:17:15,550 : INFO : topic #3 (0.011): 0.184*"859" + 0.089*"844" + 0.040*"50" + 0.036*"9" + 0.033*"652" + 0.029*"69" + 0.027*"427" + 0.025*"1169" + 0.025*"1079" + 0.024*"6"
2022-02-24 11:17:15,551 : INFO : topic #77 (0.012): 0.211*"83" + 0.182*"34" + 0.116*"35" + 0.107*"611" + 0.053*"1015" + 0.032*"69" + 0.027*"404" + 0.018*"73" + 0.017*"486" + 0.015*"131"
2022-02-24 11:17:15,551 : INFO : topic diff=0.511729, rho=0.466151
2022-02-24 11:17:15,559 : INFO : PROGRESS: pass 3, at document #2000/3204
2022-02-24 11:17:16,349 : INFO : optimized alpha [0.010161396, 0.010769062, 0.010075143, 0.011375612, 0.010023215, 0.009968814, 0.010019704, 0.010465304, 0.010806389, 0.010271793, 0.009788983, 0.010032378, 0.010146818, 0.010337888, 0.009899497, 0.009929014, 0.010341038, 0.010822783, 0.010673107, 0

2022-02-24 11:17:18,746 : INFO : topic diff=0.600594, rho=0.389191
2022-02-24 11:17:19,578 : INFO : -6.652 per-word bound, 100.6 perplexity estimate based on a held-out corpus of 1204 documents with 49783 words
2022-02-24 11:17:19,579 : INFO : PROGRESS: pass 4, at document #3204/3204
2022-02-24 11:17:20,296 : INFO : optimized alpha [0.010489356, 0.0116096, 0.010386726, 0.0122858975, 0.01027452, 0.010277755, 0.010221981, 0.0108942455, 0.011391607, 0.010660262, 0.009915718, 0.010465892, 0.010620447, 0.010583526, 0.010237601, 0.010120113, 0.010622036, 0.011520822, 0.010904235, 0.010496722, 0.010227034, 0.010590116, 0.011233913, 0.0108610345, 0.010230971, 0.010210768, 0.010056468, 0.010326413, 0.010492677, 0.010666622, 0.011067532, 0.009724096, 0.010721692, 0.010433292, 0.009954659, 0.010590065, 0.010459249, 0.010031807, 0.010170453, 0.010161653, 0.01145369, 0.010182305, 0.010470904, 0.0111770425, 0.0107740145, 0.010345899, 0.010390336, 0.0106007, 0.011239035, 0.010038362, 0.010620887, 0.0

2022-02-24 11:17:23,440 : INFO : merging changes from 2000 documents into a model of 3204 documents
2022-02-24 11:17:23,450 : INFO : topic #31 (0.010): 0.118*"3" + 0.059*"50" + 0.058*"729" + 0.039*"67" + 0.034*"725" + 0.032*"624" + 0.030*"454" + 0.026*"64" + 0.025*"73" + 0.022*"731"
2022-02-24 11:17:23,451 : INFO : topic #58 (0.010): 0.086*"50" + 0.063*"3" + 0.047*"351" + 0.036*"85" + 0.036*"518" + 0.028*"0" + 0.023*"48" + 0.022*"341" + 0.022*"304" + 0.020*"733"
2022-02-24 11:17:23,451 : INFO : topic #3 (0.013): 0.145*"859" + 0.106*"844" + 0.065*"9" + 0.040*"652" + 0.038*"56" + 0.033*"52" + 0.031*"6" + 0.031*"517" + 0.030*"50" + 0.030*"356"
2022-02-24 11:17:23,451 : INFO : topic #77 (0.015): 0.197*"83" + 0.166*"611" + 0.166*"34" + 0.118*"1015" + 0.049*"35" + 0.036*"69" + 0.024*"404" + 0.018*"315" + 0.016*"131" + 0.016*"486"
2022-02-24 11:17:23,452 : INFO : topic #91 (0.016): 0.328*"34" + 0.312*"35" + 0.194*"83" + 0.023*"254" + 0.015*"1057" + 0.011*"24" + 0.011*"1093" + 0.010*"219" + 0.

...

2022-02-24 11:17:41,381 : INFO : topic diff=0.216722, rho=0.245426
2022-02-24 11:17:42,098 : INFO : -6.479 per-word bound, 89.2 perplexity estimate based on a held-out corpus of 1204 documents with 49783 words
2022-02-24 11:17:42,098 : INFO : PROGRESS: pass 14, at document #3204/3204
2022-02-24 11:17:42,692 : INFO : optimized alpha [0.012404403, 0.0184544, 0.013146149, 0.016766494, 0.011527606, 0.012961726, 0.011719084, 0.013940027, 0.014282156, 0.013007513, 0.011092477, 0.012834636, 0.0159458, 0.012234532, 0.012309218, 0.011166613, 0.011849957, 0.015457194, 0.012141082, 0.012982884, 0.0113689685, 0.013267963, 0.014811917, 0.01203702, 0.011486702, 0.0118249, 0.011589241, 0.012123028, 0.012779561, 0.012117915, 0.013387958, 0.0104385335, 0.012929742, 0.012754691, 0.010710548, 0.013099396, 0.01207871, 0.010658882, 0.010854074, 0.0116202785, 0.013372128, 0.011351818, 0.011579622, 0.013572783, 0.012165323, 0.012279211, 0.01155838, 0.011863861, 0.015235121, 0.011479346, 0.012324064, 0.012052

2022-02-24 11:17:45,326 : INFO : merging changes from 2000 documents into a model of 3204 documents
2022-02-24 11:17:45,337 : INFO : topic #31 (0.011): 0.147*"3" + 0.066*"50" + 0.063*"729" + 0.059*"67" + 0.041*"624" + 0.041*"725" + 0.030*"216" + 0.026*"454" + 0.024*"90" + 0.023*"204"
2022-02-24 11:17:45,338 : INFO : topic #71 (0.010): 0.200*"478" + 0.114*"1103" + 0.099*"159" + 0.049*"1156" + 0.045*"0" + 0.038*"380" + 0.033*"34" + 0.032*"1159" + 0.027*"694" + 0.026*"715"
2022-02-24 11:17:45,338 : INFO : topic #1 (0.019): 0.339*"29" + 0.075*"30" + 0.055*"345" + 0.041*"13" + 0.039*"6" + 0.035*"354" + 0.030*"352" + 0.025*"50" + 0.023*"172" + 0.020*"174"
2022-02-24 11:17:45,339 : INFO : topic #77 (0.023): 0.215*"611" + 0.203*"83" + 0.161*"34" + 0.134*"1015" + 0.025*"404" + 0.025*"69" + 0.022*"486" + 0.016*"872" + 0.016*"73" + 0.016*"315"
2022-02-24 11:17:45,339 : INFO : topic #91 (0.047): 0.387*"34" + 0.376*"35" + 0.175*"83" + 0.010*"1057" + 0.008*"254" + 0.008*"386" + 0.006*"1093" + 0.005*

2022-02-24 11:17:48,515 : INFO : topic #1 (0.021): 0.361*"29" + 0.077*"30" + 0.057*"345" + 0.037*"6" + 0.036*"13" + 0.031*"354" + 0.025*"352" + 0.024*"50" + 0.024*"172" + 0.022*"174"
2022-02-24 11:17:48,516 : INFO : topic #77 (0.024): 0.227*"611" + 0.205*"83" + 0.156*"34" + 0.114*"1015" + 0.028*"404" + 0.026*"486" + 0.022*"69" + 0.017*"872" + 0.017*"73" + 0.017*"315"
2022-02-24 11:17:48,517 : INFO : topic #91 (0.054): 0.394*"34" + 0.382*"35" + 0.167*"83" + 0.011*"1057" + 0.007*"386" + 0.007*"254" + 0.006*"1093" + 0.006*"1054" + 0.004*"24" + 0.004*"36"
2022-02-24 11:17:48,517 : INFO : topic diff=0.174604, rho=0.225865
2022-02-24 11:17:48,525 : INFO : PROGRESS: pass 18, at document #2000/3204
2022-02-24 11:17:49,132 : INFO : optimized alpha [0.013070532, 0.021121815, 0.014413666, 0.018060256, 0.011882744, 0.014143735, 0.012208047, 0.015431898, 0.015197664, 0.01384762, 0.011629033, 0.013813318, 0.018423237, 0.012834917, 0.0129474895, 0.011531091, 0.012136224, 0.016588593, 0.012636794, 0.0

2022-02-24 11:17:50,994 : INFO : topic diff=0.148283, rho=0.215156
2022-02-24 11:17:51,657 : INFO : -6.448 per-word bound, 87.3 perplexity estimate based on a held-out corpus of 1204 documents with 49783 words
2022-02-24 11:17:51,657 : INFO : PROGRESS: pass 19, at document #3204/3204
2022-02-24 11:17:52,175 : INFO : optimized alpha [0.01334283, 0.022599747, 0.015098687, 0.018762976, 0.01215825, 0.014713722, 0.012495292, 0.016189048, 0.015723202, 0.014268977, 0.011949642, 0.014326525, 0.01975138, 0.013122515, 0.01326886, 0.011713084, 0.012292197, 0.017069113, 0.0129725, 0.014235037, 0.011897528, 0.014594503, 0.016390108, 0.012626469, 0.012025655, 0.012688211, 0.012564784, 0.01311468, 0.013952588, 0.012947158, 0.01441433, 0.010861633, 0.014067721, 0.013856839, 0.011184266, 0.014234861, 0.012910378, 0.011209824, 0.011231996, 0.012250764, 0.01431686, 0.011879735, 0.012133742, 0.014798154, 0.0129159875, 0.013185717, 0.012281837, 0.012478438, 0.017251994, 0.012185498, 0.013139085, 0.01315658

[(0, 0.005538163),
 (1, 0.009380399),
 (2, 0.00626696),
 (3, 0.0077878837),
 (4, 0.005046483),
 (5, 0.006107174),
 (6, 0.005186378),
 (7, 0.0067195324),
 (8, 0.0065261754),
 (9, 0.005922575),
 (10, 0.0049598967),
 (11, 0.0059464616),
 (12, 0.008198137),
 (13, 0.005446717),
 (14, 0.00550746),
 (15, 0.004861709),
 (16, 0.00510208),
 (17, 0.0070848176),
 (18, 0.005384451),
 (19, 0.005908488),
 (20, 0.004938266),
 (21, 0.00605769),
 (22, 0.0068029854),
 (23, 0.005240825),
 (24, 0.004991447),
 (25, 0.005266452),
 (26, 0.0052152216),
 (27, 0.005443465),
 (28, 0.0057912525),
 (29, 0.0053739324),
 (30, 0.005982906),
 (31, 0.0045083007),
 (32, 0.0058390405),
 (33, 0.0057515106),
 (34, 0.0046422146),
 (35, 0.0059084147),
 (36, 0.005358666),
 (37, 0.0046528233),
 (38, 0.0046620257),
 (39, 0.0050848825),
 (40, 0.0059424494),
 (41, 0.0049308804),
 (42, 0.0050363103),
 (43, 0.006142219),
 (44, 0.0053609945),
 (45, 0.00547295),
 (46, 0.00509778),
 (47, 0.0051793824),
 (48, 0.007160726),
 (49, 0.00505

\#### Please do not change this. This cell is used for grading.

---
Now we can use the `DenseRetrievalModel` class to obtain an LDA search function.
You can test your LDA model in the following cell: Try finding queries which are lexically different to documents, but semantically similar - does LDA work well for these queries?!

In [40]:
drm_lda = DenseRetrievalRanker(lda, jenson_shannon_sim)

# test your LDA model
search_fn = drm_lda.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

Searching for: 'root' (SEARCH FN: <bound method DenseRetrievalRanker.search of <__main__.DenseRetrievalRanker object at 0x00000270E68AFA00>>)


C:\Users\Bram\AppData\Local\Temp/ipykernel_15384/2120849064.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  return sum(p[x] * np.log2((p[x]) / (q[x])) for x in range(len(p)) if p[x] != 0.0 or p[x] != 0)


## Section 8: Word2Vec/Doc2Vec (20 points) <a class="anchor" id="2vec"></a>

[Back to Part 2](#part2)

We will implement two other methods here, the Word2Vec model and the Doc2Vec model, also using `gensim`. Word2Vec creates representations of words, not documents, so the word level vectors need to be aggregated to obtain a representation for the document. Here, we will simply take the mean of the vectors. 


A drawback of these models is that they need a lot of training data. Our dataset is tiny, so in addition to using a model trained on the data, we will also use a pre-trained model for Word2Vec (this will be automatically downloaded).     

*Note*:
1. The code in vectorize_documents / vectorize_query should return gensim-like vectors i.e `[(dim, val), .. (dim, val)]`. 
2. For Word2Vec: You should also handle the following two cases: (a) A word in the query is not present in the vocabulary of the model and (b) none of the words in the query are present in the model - you can return 0 scores for all documents in this case. For either of these, you can check if a `word` is present in the vocab by using `word in self.model`


In [518]:
# TODO: Implement this! (10 points)
class W2VRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        # the dimensionality of the vectors
        self.size = 100 
        self.min_count = 1
    
    def train_model(self):
        """
        Trains the W2V model
        """
        self.model = Word2Vec(self.documents, vector_size=self.size, window=2, min_count=self.min_count, workers=4)
        
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """    
        vectors={}
        for (doc_id, _), cc in zip(self.doc_repr, self.documents):
            docvec=np.zeros(self.size)
            zer=0
            for word in cc:
                if word not in self.model.wv:
                    zer=1
            if zer==0:
                docvec = self.model.wv[cc]
            tot=np.linspace(0, self.size-1,num=self.size,dtype=int)
            real=list(zip(tot, docvec[0]))
            vectors[doc_id]=real
        return vectors

    def vectorize_query(self, query):
        """
        Vectorizes the query using the W2V model
        """
        query = process_text(query, **config_2)
        for word in query:
            if word not in self.model.wv:
                return np.zeros(self.size)
        tot=np.linspace(0, self.size-1,num=self.size,dtype=int)
        query_vector = self.model.wv[query]
        real=list(zip(tot, query_vector[0]))
        return real
        
    
    
class W2VPretrainedRetrievalModel(W2VRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        self.model_name = "word2vec-google-news-300"
        self.size = 300
    
    def train_model(self):
        """
        Loads the pretrained model
        """
        self.model = g_downloader.load(self.model_name)
        
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """    
        vectors={}
        for (doc_id, _), cc in zip(self.doc_repr, self.documents):
            docvec=np.zeros(self.size)
            zer=0
            for word in cc:
                if word not in self.model:
                    zer=1
            if zer==0:
                docvec = self.model[cc]
            tot=np.linspace(0, self.size-1,num=self.size,dtype=int)
            real=list(zip(tot, docvec[0]))
            vectors[doc_id]=real
        return vectors

    def vectorize_query(self, query):
        """
        Vectorizes the query using the W2V model
        """
        query = process_text(query, **config_2)
        for word in query:
            if word not in self.model:
                return np.zeros(self.size)
        tot=np.linspace(0, self.size-1,num=self.size,dtype=int)
        query_vector = self.model[query]
        real=list(zip(tot, query_vector[0]))
        return real


w2v = W2VRetrievalModel(doc_repr_2)
w2v.train_model()

# you can now get a W2V vector for a given query in the following way:
w2v.vectorize_query("report")

2022-02-24 16:21:23,390 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-02-24 16:21:23,483 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)
2022-02-24 16:21:23,483 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)", 'datetime': '2022-02-24T16:21:23.483566', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}
2022-02-24 16:21:23,488 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2022-02-24 16:21:23,489 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (

[(0, -0.28269136),
 (1, 0.49819764),
 (2, 0.06365946),
 (3, -0.18719114),
 (4, 0.0044463915),
 (5, -0.4760814),
 (6, 0.3114989),
 (7, 0.8384582),
 (8, -0.3059881),
 (9, -0.2152389),
 (10, -0.14277785),
 (11, -0.4898834),
 (12, 0.2404292),
 (13, -0.105481125),
 (14, 0.008063824),
 (15, -0.36006474),
 (16, 0.19489853),
 (17, -0.39115587),
 (18, -0.17926194),
 (19, -0.85692096),
 (20, 0.100521505),
 (21, -0.03871558),
 (22, 0.4771609),
 (23, -0.1497572),
 (24, -0.2730095),
 (25, -0.012713571),
 (26, -0.26674286),
 (27, -0.26500133),
 (28, -0.348026),
 (29, 0.00092919543),
 (30, 0.5044258),
 (31, -0.07639225),
 (32, 0.21755947),
 (33, -0.3499114),
 (34, -0.0765402),
 (35, 0.42081532),
 (36, 0.26548785),
 (37, -0.4087433),
 (38, -0.5128143),
 (39, -0.66547436),
 (40, 0.2047444),
 (41, -0.2456076),
 (42, -0.4033521),
 (43, -0.033661615),
 (44, 0.44646418),
 (45, -0.28750977),
 (46, -0.05724067),
 (47, -0.053733986),
 (48, 0.26694825),
 (49, 0.061883844),
 (50, 0.1839782),
 (51, -0.46389553),

In [486]:
assert len(w2v.vectorize_query("report")) == 100
assert len(w2v.vectorize_query("this is a sentence that is not mellifluous")) == 100


\#### Please do not change this. This cell is used for grading.

In [487]:
w2v_pretrained = W2VPretrainedRetrievalModel(doc_repr_2)
w2v_pretrained.train_model()

# you can now get an W2V vector for a given query in the following way:
w2v_pretrained.vectorize_query("report")

2022-02-24 16:08:10,890 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-02-24 16:08:10,982 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)
2022-02-24 16:08:10,983 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)", 'datetime': '2022-02-24T16:08:10.983378', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}
2022-02-24 16:08:11,029 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2022-02-24 16:08:11,029 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (

[(0, -0.14257812),
 (1, -0.1640625),
 (2, -0.09033203),
 (3, -0.11230469),
 (4, 0.100097656),
 (5, -0.041259766),
 (6, 0.048828125),
 (7, -0.13671875),
 (8, 0.19628906),
 (9, -0.13476562),
 (10, -0.017578125),
 (11, 0.032226562),
 (12, 0.095214844),
 (13, -0.10595703),
 (14, -0.16992188),
 (15, 0.041015625),
 (16, -0.26367188),
 (17, -0.0063171387),
 (18, -0.17773438),
 (19, -0.24023438),
 (20, 0.3515625),
 (21, -0.012207031),
 (22, -0.16210938),
 (23, -0.12060547),
 (24, 0.04321289),
 (25, 0.10986328),
 (26, 0.052490234),
 (27, 0.17871094),
 (28, -0.14550781),
 (29, 0.13769531),
 (30, -0.08203125),
 (31, -0.28320312),
 (32, -0.10888672),
 (33, -0.2890625),
 (34, 0.072265625),
 (35, -0.04736328),
 (36, 0.040283203),
 (37, 0.067871094),
 (38, 0.11669922),
 (39, 0.000831604),
 (40, 0.068359375),
 (41, 0.12011719),
 (42, -0.088378906),
 (43, 0.33789062),
 (44, -0.044677734),
 (45, -0.030151367),
 (46, 0.0076904297),
 (47, -0.021118164),
 (48, -0.25390625),
 (49, 0.14941406),
 (50, 0.39843

In [223]:
##### Function check

print(len(w2v_pretrained.vectorize_query("report")))
#####

300


In [224]:
drm_w2v = DenseRetrievalRanker(w2v, cosine_sim)

# test your LDA model
search_fn = drm_w2v.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

Searching for: 'great' (SEARCH FN: <bound method DenseRetrievalRanker.search of <__main__.DenseRetrievalRanker object at 0x00000270DEBB7190>>)


In [225]:
drm_w2v_pretrained = DenseRetrievalRanker(w2v_pretrained, cosine_sim)

#test your LDA model
search_fn = drm_w2v_pretrained.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

Searching for: 'report' (SEARCH FN: <bound method DenseRetrievalRanker.search of <__main__.DenseRetrievalRanker object at 0x00000271CCEA6340>>)


**Implementation (10 points):**
For Doc2Vec, you will need to create a list of `TaggedDocument` instead of using the `self.corpus` or `self.documents` variable. Use the document id as the 'tag'.
  

In [82]:
#### Please do not change this. This cell is used for grading.

\#### Please do not change this. This cell is used for grading.

In [582]:
# TODO: Implement this! (10 points)
class D2VRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        self.vector_size= 100
        self.min_count = 1
        self.epochs = 20
        self.doc = [TaggedDocument(d[1], d[0]) for i, d in enumerate(doc_repr)]
        
    def train_model(self):
        """
        Trains the W2V model
        """
        self.model = Doc2Vec(self.doc, vector_size=self.vector_size, window=5, min_count=self.min_count, workers=4)
        
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """    
        vectors={}
        
        for (_, doc_id) in self.doc:
            if doc_id not in self.model.dv:
                docvec=np.zeros(self.vector_size)
            else:
                docvec=self.model.dv[doc_id]
            tot=np.linspace(0, self.vector_size-1,num=self.vector_size,dtype=int)
            real=list(zip(tot, docvec))
            vectors[doc_id]=real
            
        return vectors

    def vectorize_query(self, query):
        """
        Vectorizes the query using the W2V model
        """
        query = process_text(query, **config_2)
        for word in query:
            if word not in self.model.wv:
                return np.zeros(self.vector_size)
        tot=np.linspace(0, self.vector_size-1,num=self.vector_size,dtype=int)
        query_vector = self.model.wv[query]
        real=list(zip(tot, query_vector[0]))
        
        return real
        
d2v = D2VRetrievalModel(doc_repr_2)
d2v.train_model()


# # you can now get an LSI vector for a given query in the following way:
d2v.vectorize_query("report")

2022-02-24 16:50:48,517 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-02-24 16:50:48,609 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)
2022-02-24 16:50:48,610 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115969 corpus positions)", 'datetime': '2022-02-24T16:50:48.610017', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}
2022-02-24 16:50:48,615 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2022-02-24 16:50:48,615 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (

2022-02-24 16:50:50,886 : INFO : Doc2Vec lifecycle event {'msg': 'training on 1159690 raw words (1040774 effective words) took 2.1s, 496468 effective words/s', 'datetime': '2022-02-24T16:50:50.886794', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'train'}
2022-02-24 16:50:50,887 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d100,n5,w5,s0.001,t4)', 'datetime': '2022-02-24T16:50:50.887791', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}


[(0, -0.31818688),
 (1, -0.3642748),
 (2, -0.4794853),
 (3, 0.28377432),
 (4, 0.033227865),
 (5, -0.07386055),
 (6, -0.38798797),
 (7, -0.018074824),
 (8, -0.7126169),
 (9, 0.05327832),
 (10, 0.09299619),
 (11, 0.035904825),
 (12, -0.13017577),
 (13, -0.18614832),
 (14, -0.24202296),
 (15, -0.46552107),
 (16, 0.2416039),
 (17, 0.46236992),
 (18, -0.21027808),
 (19, -0.26402554),
 (20, -0.085892364),
 (21, 0.07353525),
 (22, -0.15807979),
 (23, 0.35701814),
 (24, 0.10416707),
 (25, -0.3896972),
 (26, -0.5616182),
 (27, -0.57713115),
 (28, 0.14533271),
 (29, -0.73964125),
 (30, 0.35103235),
 (31, 0.3870823),
 (32, -0.25085452),
 (33, -0.03265667),
 (34, -0.10750699),
 (35, 0.22283392),
 (36, -0.31515235),
 (37, -0.69298786),
 (38, -0.20739423),
 (39, -0.19842733),
 (40, 0.15136081),
 (41, -0.32502896),
 (42, 0.1928304),
 (43, -0.43657002),
 (44, -0.04799908),
 (45, -0.20189176),
 (46, -0.09227677),
 (47, 0.0713862),
 (48, 0.17575511),
 (49, -0.30014163),
 (50, -0.369882),
 (51, -0.091093

In [583]:
drm_d2v = DenseRetrievalRanker(d2v, cosine_sim)

# test your LDA model
search_fn = drm_d2v.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

Searching for: 'report' (SEARCH FN: <bound method DenseRetrievalRanker.search of <__main__.DenseRetrievalRanker object at 0x000002713F701E50>>)


C:\Users\Bram\AppData\Local\Temp/ipykernel_15384/1313106104.py:23: RuntimeWarning: invalid value encountered in double_scalars
  return d/(np.dot(squar_1,squar_2))


---
## Section 9: Re-ranking (10 points) <a class="anchor" id="reranking"></a>

[Back to Part 2](#part2)

To motivate the re-ranking perspective (i.e retrieve with lexical method + rerank with a semantic method), let's search using semantic methods and compare it to BM25's performance, along with their runtime:


In [255]:
query = "algebraic functions"
print("BM25: ")
%timeit bm25_search(query, 2)
print("LSI: ")
%timeit drm_lsi.search(query)
print("LDA: ")
%timeit drm_lda.search(query)
print("W2V: ")
%timeit drm_w2v.search(query)
print("W2V(Pretrained): ")
%timeit drm_w2v_pretrained.search(query)
print("D2V:")
%timeit drm_d2v.search(query)

BM25: 
1.01 ms ± 5.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
LSI: 
1.23 s ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
LDA: 


C:\Users\Bram\AppData\Local\Temp/ipykernel_15384/2120849064.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  return sum(p[x] * np.log2((p[x]) / (q[x])) for x in range(len(p)) if p[x] != 0.0 or p[x] != 0)


KeyboardInterrupt: 

In [570]:
query = "algebraic functions"
print("BM25: ")
%timeit bm25_search(query, 2)
print("LSI: ")
%timeit drm_lsi.search(query)
#print("LDA: ")
#%timeit drm_lda.search(query)
print("W2V: ")
%timeit drm_w2v.search(query)
print("W2V(Pretrained): ")
%timeit drm_w2v_pretrained.search(query)
print("D2V:")
%timeit drm_d2v.search(query)

BM25: 


KeyboardInterrupt: 

---

**Implementation (10 points):**
Re-ranking involves retrieving a small set of documents using simple but fast methods like BM25 and then re-ranking them with the aid of semantic methods such as LDA or LSI. Implement the following class, which takes in an `initial_retrieval_fn` - the initial retrieval function and `vsrm` - an instance of the `VectorSpaceRetrievalModel` class (i.e LSI/LDA) as input. The search function should first retrieve an initial list of K documents, and then these documents are re-ranked using a semantic method. This not only makes retrieval faster, but semantic methods perform poorly when used in isolation, as you will find out.

In [597]:
# TODO: Implement this! (10 points)
def Sort_Tuple(tup): 
      
    # getting length of list of tuples
    lst = len(tup) 
    for i in range(0, lst): 
          
        for j in range(0, lst-i-1): 
            if (tup[j][1] > tup[j + 1][1]): 
                temp = tup[j] 
                tup[j]= tup[j + 1] 
                tup[j + 1]= temp 
    return tup 


class DenseRerankingModel:
    def __init__(self, initial_retrieval_fn, vsrm, similarity_fn):
        """
            initial_retrieval_fn: takes in a query and returns a list of [(doc_id, score)] (sorted)
            vsrm: instance of `VectorSpaceRetrievalModel`
            similarity_fn: function instance that takes in two vectors 
                            and returns a similarity score e.g cosine_sim defined earlier
        """
        self.ret = initial_retrieval_fn
        self.vsrm = vsrm
        self.similarity_fn = similarity_fn
        self.vectorized_documents = vsrm.vectorize_documents()
        
        assert len(self.vectorized_documents) == len(doc_repr_2)
    
    def search(self, query, K=50):
        """
            First, retrieve the top K results using the retrieval function
            Then, re-rank the results using the VSRM instance
        """
        reranked_l=[]
        for i in self.ret(query):
            query_vector=self.vsrm.vectorize_query(query)
            doc_id=i[0]
            if np.array(self.vectorized_documents[doc_id]).size==0 or  np.array(query_vector).size==0:
                continue
            else:
                

                scor=self.similarity_fn(self.vectorized_documents[doc_id],query_vector)
                reranked_l.append([doc_id,scor])
        reranked_l=Sort_Tuple(reranked_l)
        print(reranked_l)
        return reranked_l
            

In [598]:
##### Function check
bm25_search_2 = partial(bm25_search, index_set=2)
lsi_rerank = DenseRerankingModel(bm25_search_2, lsi, cosine_sim)
lda_rerank = DenseRerankingModel(bm25_search_2, lda, jenson_shannon_sim)
w2v_rerank = DenseRerankingModel(bm25_search_2, w2v, cosine_sim)
w2v_pretrained_rerank = DenseRerankingModel(bm25_search_2, w2v_pretrained, cosine_sim)
d2v_rerank = DenseRerankingModel(bm25_search_2, d2v, cosine_sim)

##### 

In [340]:
print(lsi_rerank.ret('root'))

[['1667', 7.449153363679816], ['1893', 7.449153363679816], ['5', 6.903649986863807], ['42', 6.763441377985474], ['220', 6.763441377985474], ['1031', 6.6998305979075266], ['2', 6.628814497853222], ['35', 6.628814497853222], ['194', 6.628814497853222], ['2234', 6.499442537265385], ['2440', 6.499442537265385], ['2458', 6.499442537265385], ['1440', 6.462865832192601], ['1633', 6.440679133769386], ['1524', 6.380318240404117], ['304', 6.375023711453946], ['334', 6.375023711453946], ['348', 6.375023711453946], ['503', 6.255278909861246], ['640', 6.255278909861246], ['1803', 6.255278909861246], ['2291', 6.139949605904734], ['1999', 6.087480752087412], ['788', 6.028795993783186], ['1520', 6.028795993783186], ['1567', 5.978201840572855], ['1450', 5.92159532406452], ['1763', 5.92159532406452], ['1882', 5.92159532406452], ['1615', 5.872777162932156], ['1618', 5.818140413745135], ['1790', 5.818140413745135], ['848', 5.771006346231923], ['2213', 5.718238309810039], ['380', 5.624796109253045], ['1932

\#### Please do not change this. This cell is used for grading.

---
Now, let us time the new search functions:

In [573]:
query = "algebraic functions"
print("BM25: ")
%timeit bm25_search(query, 2)
print("LSI: ")
%timeit lsi_rerank.search(query)
print("LDA: ")
%timeit lda_rerank.search(query)
print("W2V: ")
%timeit w2v_rerank.search(query)
print("W2V(Pretrained): ")
%timeit w2v_pretrained_rerank.search(query)
print("D2V:")
%timeit d2v_rerank.search(query)

BM25: 


KeyboardInterrupt: 

---
As you can see, it is much faster (but BM25 is still orders of magnitude faster).

---
## Section 10: Evaluation & Analysis (30 points) <a class="anchor" id="reranking_eval"></a>

[Back to Part 2](#part2)

[Previously](#evaluation) we have implemented some evaluation metrics and used them for measuring the ranking performance of term-based IR algorithms. In this section, we will do the same for semantic methods, both with and without re-ranking.

### Section 10.1: Plot (10 points)

First, gather the results. The results should consider the index set, the different search functions and different metrics. Plot the results in bar charts, per metric, with clear labels.

Then, gather only the re-ranking models, and plot and compare them with the results obtained in part 1 (only index set 2).

In [261]:
list_of_sem_search_fns = [
    ("lda", drm_lda.search),
    ("lsi", drm_lsi.search),
    ("w2v", drm_w2v.search),
    ("w2v_pretrained", drm_w2v_pretrained.search),
    ("d2v", drm_d2v.search),
    ("lsi_rr", lsi_rerank.search),
    ("lda_rr", lda_rerank.search),
    ("w2v_rr", w2v_rerank.search),
    ("w2v_pretrained_rr", w2v_pretrained_rerank.search),
    ("d2v_rr", d2v_rerank.search),
    
]

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Section 10.2: Summary (20 points)

Your summary should compare methods from Part 1 and Part 2 (only for index set 2). State what you expected to see in the results, followed by either supporting evidence *or* justify why the results did not support your expectations. Consider the availability of data, scalability, domain/type of data, etc.

YOUR ANSWER HERE